# Настройка Colab
Выполним подключение к Google Диску для доступа к файлам .py и данным.

In [ ]:
# открытие окна с запросом о предоставлении разрешений
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# укажите путь на свой google-диск
# в данном случае папка с материалами лабы Lab_GAN расположена в корне google-диска
import os
datadir = "/content/Lab_GAN"
if not os.path.exists(datadir):
  !ln -s "/content/drive/MyDrive/Lab_GAN" $datadir # TODO: замените на свой путь
os.chdir(datadir)
!pwd

# Генеративные состязательные сети (Generative Adversarial Networks)
Сначала необходимо реализовать два разных типа генеративных состязательных сетей. Будем обучаться на базе изображений кошек.

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # установим размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [ ]:
from gan.train import train

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Функции потерь GAN (GAN loss functions)

Необходимо реализовывать две разные функции потерь GAN: как в [оригинальной статье о GAN](http://arxiv.org/pdf/1406.2661.pdf) и как в [LS-GAN](https://arxiv.org/abs/1611.04076).

### Функции потерь GAN

**TODO:** Реализуйте функции `discriminator_loss` и `generator_loss` in файле `gan/losses.py`.

Функция потерь генератора определяется как:
$$\ell_G  =  -\mathbb{E}_{z \sim p(z)}\left[\log D(G(z))\right]$$
и функция потерь дискриминатора равна:
$$ \ell_D = -\mathbb{E}_{x \sim p_\text{data}}\left[\log D(x)\right] - \mathbb{E}_{z \sim p(z)}\left[\log \left(1-D(G(z))\right)\right]$$

Обратите внимание на знак минус в уравнениях, поскольку мы будем *минимизировать* эти функции потерь.



**HINTS**: Следует использовать функцию `torch.nn.functional.binary_cross_entropy_with_logits` для вычисления бинарной кросс-энтропии, поскольку эта функция численно более стабильна, чем использование softmax, за которой следует BCE.




Пусть оценка (score) $s\in\mathbb{R}$ и метка $y\in\{0, 1\}$, бинарная кросс-энтропия равна

$$ bce(s, y) = -y * \log(s) - (1 - y) * \log(1 - s) $$

Тогда
$$\ell_G = bce(D(G(z)), 1)$$

Первое слагаемое в $\ell_D$ равно
$$bce(D(x), 1)$$


Второе слагаемое в $\ell_D$ равно
$$bce(1-D(G(z)), 1) = bce(D(G(z)), 0)$$

Вместо вычисления математического ожидания для $\log D(G(z))$, $\log D(x)$ и $\log \left(1-D(G(z))\right)$, будем выполнять усреднение по элементам минибатча, поэтому убедитесь, что объединяете потери путем усреднения, а не суммирования.

In [ ]:
from gan.losses import discriminator_loss, generator_loss

### Least Squares GAN loss (функция потерь GAN по методу наименьших квадратов)

**TODO:** Реализуйте функции `ls_discriminator_loss` и `ls_generator_loss` в файле `gan/losses.py`.


Теперь рассмотрим [Least Squares GAN](https://arxiv.org/abs/1611.04076), более новую и стабильную альтернативу исходной функции потерь GAN. Все, что нужно сделать, это изменить функцию потерь и переобучить модель. Функция потерь генератора для этого случая равна:


$$\ell_G  =  \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[\left(D(G(z))-1\right)^2\right]$$
функция потерь дискриминатора:
$$ \ell_D = \frac{1}{2}\mathbb{E}_{x \sim p_\text{data}}\left[\left(D(x)-1\right)^2\right] + \frac{1}{2}\mathbb{E}_{z \sim p(z)}\left[ \left(D(G(z))\right)^2\right]$$



**HINTS**: Вместо вычисления математического ожидания мы будем выполнять усреднение по элементам минибатча, поэтому не забудьте объединить потери путем усреднения, а не суммирования. При подстановке значений для $D(x)$ и $D(G(z))$ используйте непосредственно выходные значения дискриминатора (`scores_real` и `scores_fake`).

In [ ]:
from gan.losses import ls_discriminator_loss, ls_generator_loss

# Архитектура модели GAN

**TODO:** Реализуйте классы the `Discriminator` и `Generator` в файле `gan/models.py`.

Рекомендуется использовать следующую архитектуру, вдохновленную [DCGAN](https://arxiv.org/pdf/1511.06434.pdf):

**Дискриминатор:**

- сверточный слой (in_channels=3, out_channels=128, kernel_size=4, stride=2, padding=1)
- сверточный слой (in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1)
- батч-нормализация
- сверточный слой (in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1)
- батч-нормализация
- сверточный слой (in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1)
- батч-нормализация
- сверточный слой (in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0)


Вместо ReLU применим LeakyReLU (negative slope зададим равным 0.2). Можно выбрать просто ReLU.


На выходе дискриминатора должен быть вектор оценок, каждая из которых соответствует своему входному примеру.


**Генератор:**

**Примечание:** В генераторе вам нужно будет использовать транспонированную свертку (transposed convolution). Эта функция реализована в pytorch как `torch.nn.ConvTranspose2d`.


- транспонированная свертка (in_channels=NOISE_DIM, out_channels=1024, kernel=4, stride=1, padding=0)
- батч-нормализация
- транспонированная свертка (in_channels=1024, out_channels=512, kernel=4, stride=2, padding=1)
- батч-нормализация
- транспонированная свертка (in_channels=512, out_channels=256, kernel=4, stride=2, padding=1)
- батч-нормализация
- транспонированная свертка (in_channels=256, out_channels=128, kernel=4, stride=2, padding=1)
- батч-нормализация
- транспонированная свертка (in_channels=128, out_channels=3, kernel=4, stride=2, padding=1)


Используйте функцию активации ReLU в генераторе (см. [оригинальную статью DCGAN](https://arxiv.org/pdf/1511.06434.pdf)).
Выход последнего слоя генератора должен иметь нелинейность `tanh`, чтобы выводить значения от -1 до 1. Выходные данные должны представлять собой тензор размером 3x64x64 для каждого сгенерированного примера (его размеры должны быть равны размеру изображений из набора данных).




In [ ]:
from gan.models import Discriminator, Generator

# Загрузка данных

Изображения кошек являются RGB-изображениями с разрешением 64x64. Чтобы предотвратить переобучение дискриминатора, необходимо применить аугментацию данных.

**TODO:** Реализуйте аугментацию данных, добавив новые преобразования в ячейку ниже. Как минимум, необходимо использовать RandomCrop и ColorJitter, но интересно применить другие аугментации и проанализировать, как изменится качество работы GAN. https://pytorch.org/vision/stable/transforms.html.


In [ ]:
# распаковка архива в хранилище сеанса для последующего ускорения чтения изображений
!unzip "/content/drive/MyDrive/Lab_GAN/cats.zip" -d "/content"

In [ ]:
# Создание Dataset и DataLoader
batch_size = 64
imsize = 64

cat_root = '/content/cats'

cat_train = ImageFolder(root=cat_root, transform=transforms.Compose([
  transforms.ToTensor(),

  # Пример использования RandomCrop:
  transforms.Resize(int(1.15 * imsize)),
  transforms.RandomCrop(imsize),
]))

cat_loader_train = DataLoader(cat_train, batch_size=batch_size, drop_last=True)

### Визуализация датасета

In [ ]:
from gan.utils import show_images

try:
  imgs = next(iter(cat_loader_train))[0].numpy().squeeze()
except:
  imgs = cat_loader_train.__iter__().next()[0].numpy().squeeze()

show_images(imgs, color=True)

# Обучение

**TODO:** Допишите недостающие строки в цикле обучения в файле `gan/train.py`.

In [ ]:
NOISE_DIM = 100
NUM_EPOCHS = 50
learning_rate = 0.001

### Обучение GAN

In [ ]:
D = Discriminator().to(device)
G = Generator(noise_dim=NOISE_DIM).to(device)

In [ ]:
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas = (0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas = (0.5, 0.999))

In [ ]:
# original gan
train(D, G, D_optimizer, G_optimizer, discriminator_loss,
          generator_loss, num_epochs=NUM_EPOCHS, show_every=250,
          batch_size=batch_size, train_loader=cat_loader_train, device=device)


### Обучение LS-GAN

In [ ]:
D = Discriminator().to(device)
G = Generator(noise_dim=NOISE_DIM).to(device)

In [ ]:
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas = (0.5, 0.999))
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas = (0.5, 0.999))

In [ ]:
# ls-gan
train(D, G, D_optimizer, G_optimizer, ls_discriminator_loss,
          ls_generator_loss, num_epochs=NUM_EPOCHS, show_every=250,
          batch_size=batch_size, train_loader=cat_loader_train, device=device)